In 05_CNN, we attached Dense layers to a pretrained model. We trained only the new layers. 
It is a good idea to fine tune last few layers of trained model itself. This is what we will 
do in this notebook

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Load the base model

In [2]:
from keras.applications import VGG16

Using TensorFlow backend.


In [3]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(150,150,3))

In [4]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [5]:
for layer in base_model.layers:
    print (layer.name, layer.trainable)

('input_1', False)
('block1_conv1', True)
('block1_conv2', True)
('block1_pool', True)
('block2_conv1', True)
('block2_conv2', True)
('block2_pool', True)
('block3_conv1', True)
('block3_conv2', True)
('block3_conv3', True)
('block3_pool', True)
('block4_conv1', True)
('block4_conv2', True)
('block4_conv3', True)
('block4_pool', True)
('block5_conv1', True)
('block5_conv2', True)
('block5_conv3', True)
('block5_pool', True)


In [6]:
base_model.trainable = False

In [7]:
for layer in base_model.layers:
    print (layer.name, layer.trainable)

('input_1', False)
('block1_conv1', True)
('block1_conv2', True)
('block1_pool', True)
('block2_conv1', True)
('block2_conv2', True)
('block2_pool', True)
('block3_conv1', True)
('block3_conv2', True)
('block3_conv3', True)
('block3_pool', True)
('block4_conv1', True)
('block4_conv2', True)
('block4_conv3', True)
('block4_pool', True)
('block5_conv1', True)
('block5_conv2', True)
('block5_conv3', True)
('block5_pool', True)


In [8]:
# For some reason "conv_base_model.trainable = False" does not set "layer.trainable=False" 
# for any layer

base_model.trainable = False

time_to_set_trainable = False

for layer in base_model.layers:
    
    if layer.name == 'block5_conv1':
        time_to_set_trainable = True
        
    layer.trainable = time_to_set_trainable

In [9]:
for layer in base_model.layers:
    print (layer.name, layer.trainable)

('input_1', False)
('block1_conv1', False)
('block1_conv2', False)
('block1_pool', False)
('block2_conv1', False)
('block2_conv2', False)
('block2_pool', False)
('block3_conv1', False)
('block3_conv2', False)
('block3_conv3', False)
('block3_pool', False)
('block4_conv1', False)
('block4_conv2', False)
('block4_conv3', False)
('block4_pool', False)
('block5_conv1', True)
('block5_conv2', True)
('block5_conv3', True)
('block5_pool', True)


## add new layers on top

In [10]:
from keras import Sequential
from keras.layers import Dense, Flatten

In [11]:
model = Sequential()

model.add(base_model)

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [19]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

### set up data sources

In [20]:
import os

In [21]:
root_dir = "/"
users = os.path.join(root_dir, "Users")
airwoot = os.path.join(users, "airwoot")
documents = os.path.join(airwoot, "Documents")
anuj = os.path.join(documents, "Anuj")
Warehouse = os.path.join(anuj, "Warehouse")


src_folder = os.path.join(Warehouse, "train")

cat_src_folder = os.path.join(src_folder, "CAT")
dog_src_folder = os.path.join(src_folder, "DOG")

dest_data_folder = os.path.join(Warehouse, "cats_and_dogs")

train_dir = os.path.join(dest_data_folder, "train")
cat_train_dir = os.path.join(train_dir, "cat")
dog_train_dir = os.path.join(train_dir, "dog")


test_dir = os.path.join(dest_data_folder, "test")
cat_test_dir = os.path.join(test_dir, "cat")
dog_test_dir = os.path.join(test_dir, "dog")


validation_dir = os.path.join(dest_data_folder, "validation")
cat_validation_dir = os.path.join(validation_dir, "cat")
dog_validation_dir = os.path.join(validation_dir, "dog")

In [22]:
## stream the data

In [23]:
from keras.preprocessing.image import ImageDataGenerator

In [24]:
train_IDG = ImageDataGenerator(rescale=1.0/255, 
                               vertical_flip=True, 
                               horizontal_flip=True,
                              )

train_DG = train_IDG.flow_from_directory(train_dir,
                                         target_size=(150,150),
                                         class_mode='binary',
                                         batch_size=20)
                            
             
validation_IDG = ImageDataGenerator(rescale=1.0/255)
                               

valid_DG = validation_IDG.flow_from_directory(validation_dir,
                                         target_size=(150,150),
                                         class_mode='binary',
                                         batch_size=20)

Found 4000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [25]:
history = model.fit_generator(train_DG, epochs=2, steps_per_epoch=5, validation_data=valid_DG)

Epoch 1/2


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
    
plt.show()